# Deep Learning - Exercise 8

The aim of this exercise is to learn how to implement and utilize attention mechanisms in deep learning models, focusing on how these techniques allow models to selectively focus on the most relevant parts of input data.

**Core Concepts**
* 🧠 Attention mechanism fundamentals and mathematical foundations
* 🔍 Types of attention mechanisms (Self-attention, Dot-product)
* 📊 Applications in natural language processing
* ⚙️ Implementation of attention-based models

We recommend reading [this post](https://analyticsindiamag.com/a-beginners-guide-to-using-attention-layer-in-neural-networks/) for more detailed information.

[Open in Google colab](https://colab.research.google.com/github/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_08.ipynb)
[Download from Github](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_08.ipynb)

##### Remember to set **GPU** runtime in Colab!

In [ ]:
# !pip install keract
!pip install attention

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow import string as tf_string
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Layer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import os

# from keract import get_activations
from keras import Input, Model
from tensorflow.keras.callbacks import Callback
import keras.backend as K

os.environ['KERAS_ATTENTION_DEBUG'] = '1'
from attention import Attention

plt.rcParams.update({'font.size': 8})

keras.utils.set_random_seed(13)
tf.version.VERSION

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

# 📒 What is the Attention mechanism?

* When we think about the English word **Attention**, we know that it means **directing your focus at something** and taking greater notice
* The Attention mechanism in Deep Learning is based off this concept of directing your focus, and it pays greater attention to certain factors when processing the data
    * 📌 Paying attention to important information is necessary and it can improve the performance of the model
* **Attention mechanism can help a neural network to memorize long sequences of the information**
    * 🔎 Remember the RNN and even LSTM long-context issues?
* 🔎 Can you imagine some use-cases where it can help us?

> 💡 In very simple terms the Attention mechanism makes sure that the forget mechanism of LSTM layers is not applied over the important pieces of information

### The process is usually computed in these few steps

![Img00](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_08_04.png?raw=true)

* Let’s say that we have an input with n sequences and output y with m sequence in a network
    * $x=[x_1, x_2, ..., x_n]$
    * $y = [y_1, y_2, ..., y_n]$
    
* The encoder which we are using in the network is a bidirectional LSTM network where it has a forward hidden state and a backward hidden state
    * Representation of the encoder state can be done by concatenation of these forward and backward states
    * $h_i = [h_i^{L2R}, h_i^{R2L}]$

* The hidden state is:
    * $s_t=f(s_{t-1}, y_{t-1}, c_t)$
    
* For the output word at position t, the context vector $C_t$ can be the sum of the hidden states of the input sequence
* Thus we have:

![Img02](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_08_02.png?raw=true)

* Here we can see that the sum of the hidden state is weighted by the alignment scores
* 💡 We can say that ${\alpha_{t,i}}$  are the weights that are responsible for defining how much of each source’s hidden state should be taken into consideration for each output

* 💡 There can be various types of alignment scores according to their geometry
    * It can be either linear or in the curve geometry

### 📌 Below are some of the popular attention mechanisms:

![Img03](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_08_03.png?raw=true)

#### 💡 There are many variants of the mechanism in the wild but the basic computation process is the same

### The very common and easy to understand example is **Self-Attention Mechanism**
* When an attention mechanism is applied to the network so that it can relate to different positions of a single sequence and can compute the representation of the same sequence, it can be considered as self-attention

![Img01](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_08_01.png?raw=true)

* Here in the image, the red color represents the word which is currently learning and the blue color is of the memory, and the intensity of the color represents the degree of memory activation

## ⚡ We will use Attention layer from the library first and try to solve the *Find-Max task*
* What is the *Find-Max* task?

In [ ]:
def generate_data(num_samples, seq_length):
    X = np.random.uniform(0, 10, (num_samples, seq_length, 1))
    y = np.max(X, axis=1)  # Find maximum value in each sequence
    return X, y

seq_length = 15
# Generate 10,000 sequences of length 15
X_train, y_train = generate_data(10000, seq_length)
X_test, y_test = generate_data(1000, seq_length)

## ⚡ We will employ a simple LSTM-based model with attention layer stacked to it
* 🔎 What is the intuition behind using attention?

In [ ]:
inputs = Input(shape=(seq_length, 1))
x = LSTM(64, return_sequences=True)(inputs)
x = Attention(units=32)(x)  # Bahdanau-style additive attention
outputs = Dense(1)(x)
model = Model(inputs, outputs)
model.compile(loss='mse', optimizer='adam')
model.summary()

## We want to access the attention weights so we need to create a custom model to access it

In [ ]:
model.layers

In [ ]:
visualization_model = Model(
    inputs=inputs,
    outputs=[model.layers[-1].output, model.layers[4].output]  # Access attention layer
)

# 🚀 Let's train the model

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=40,
    batch_size=32
)

## Now we can predict the values for the testing set

In [ ]:
preds = model.predict(X_test)
preds.ravel()[:10], y_test.ravel()[:10]

## 💡 How should the predicted vs. true values scatter plot look like?

In [ ]:
sns.scatterplot(x=y_test.ravel(), y=preds.ravel())

In [ ]:
# Test with sample sequence
prediction_arr, attention_weights_arr = [], []
for i in range(12):
    prediction, attention_weights = visualization_model.predict(X_test[i])
    prediction_arr.append(prediction)
    attention_weights_arr.append(attention_weights)
# prediction, attention_weights = visualization_model.predict(X_test[0])

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(18, 15))
gs = gridspec.GridSpec(4, 3)

for i in range(12):
    ax = plt.subplot(gs[i//3, i%3])
    ax.plot(X_test[i], 'b-o', label='Sequence Values')
    ax.plot(attention_weights_arr[i], 'r--s', label='Attention Weights')
    ax.set_title(f'Max = {y_test[i][0]:.2f}, MaxPos = {np.argmax(X_test[i])}, AttMaxPos = {np.argmax(attention_weights_arr[i])}')
    ax.set_xlabel('Sequence Position')
    if i % 3 == 0:
        ax.set_ylabel('Value / Weight')
    if i == 0:
        ax.legend()

plt.tight_layout()
plt.show()

## ⚡ Now as we know how the Attention layer works we can employ it for the sentiment analysis task
* We will use Yelp dataset which contains reviews of restaurants with either positive (1) or negative (0) labels assigned

## Download and load the dataset

In [ ]:
path_to_file = tf.keras.utils.get_file('yelp_labelled.txt', 'https://raw.githubusercontent.com/lubsar/VSB-FEI-Deep-Learning-Exercises/main/datasets/yelp_labelled.txt')

In [ ]:
path_to_file

In [ ]:
with open(path_to_file) as f:
    lines = f.readlines()
    lines = [x.rstrip() for x in lines]

In [ ]:
len(lines)

In [ ]:
lines_dict = [{'Text': x[:-1].rstrip(), 'Label': int(x[-1])} for x in lines]

In [ ]:
df = pd.DataFrame.from_dict(lines_dict)

In [ ]:
df.head()

### ⚡ We will use TextVectorization layer as usuall and we will create baseline model without the Attention layer first

In [ ]:
embedding_dim = 64 # Dimension of embedded representation - this is already part of latent space, there is captured some dependecy among words, we are learning this vectors in ANN
max_tokens = 3000
sequence_length = 32 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(df.Text.values)

In [ ]:
vocab = vect_layer.get_vocabulary()

##  The dataset is balanced
* 💡 We will use `stratify` parameter of the `train_test_split` to make sure that it stays balanced

In [ ]:
df.Label.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.Text, df.Label, test_size=0.20, random_state=13, stratify=df.Label)

Fix string type

In [ ]:
X_train = X_train.astype(str).to_numpy()
X_test = X_test.astype(str).to_numpy()

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
print('Train')
print(y_train.value_counts())
print('Test')
print(y_test.value_counts())

## Let's define very simple model first

This LSTM layer processes sequential data from the preceding embedding layer (`emb`), generates a 50-dimensional output vector, and applies two types of dropout regularization to combat overfitting


### Key Components of LSTM
1. **Units (50)**  
   - The LSTM layer contains 50 memory cells, each capable of learning long-term dependencies in sequential data
   - The final output (`x`) is a 50-dimensional vector summarizing the sequence’s contextual information

2. **Dropout (0.3)**  
   - **Input Dropout**: Randomly drops 30% of input units during training to prevent overfitting
   - Example: If the embedding layer outputs 300-dimensional vectors, 90 features (30% of 300) are masked in each training step 

3. **Recurrent Dropout (0.4)**  
   - Drops 40% of the recurrent connections (hidden state transitions between timesteps) during training
   - This specifically regularizes the LSTM’s internal memory mechanisms

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(len(vocab), output_dim=embedding_dim, embeddings_regularizer=keras.regularizers.l2(.001))(x_v)
x = LSTM(50, dropout=0.3, recurrent_dropout=0.4)(emb)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer=keras.optimizers.AdamW(), loss=keras.losses.BinaryCrossentropy(from_logits=False), metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best.weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

In [ ]:
batch_size = 128
epochs = 20

history = model.fit(X_train, y_train.values, validation_split=0.2, callbacks=[model_checkpoint_callback], epochs=epochs, batch_size=batch_size)

show_history(history)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test.values)
test_acc

# Now we will create our own Attention layer and add it to the model

![meme01](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_08_meme_01.jpg?raw=true)



In [ ]:
class MyAttention(Layer):
    def __init__(self, **kwargs):
        super(MyAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")        
        super(MyAttention, self).build(input_shape)

    def call(self, x):
        dot = tf.matmul(x, self.W) + self.b
        th = tf.math.tanh(dot)
        et = tf.squeeze(th, axis=-1) # Remove last dimension, similar to np.squeeze
        at = tf.nn.softmax(et)
        at = tf.expand_dims(at, axis=-1) # Add last dimension, similar to np.expand_dims
        output = x * at
        res = tf.reduce_sum(output, axis=1) # Sum along the sequence length
        return res 

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        return super(MyAttention, self).get_config()


## 💡 The shapes for our layer will look like this:
* input (None, 32, 128) W: (128, 1) b: (32, 1)
* dot (None, 32, 1)
* th (None, 32, 1)
* squeeze (None, 32)
* softmax (None, 32)
* expand_dims (None, 32, 1)
* output (None, 32, 128)
* res (None, 128)

### 🔎 Why do we have `32` biases and `128` weights?

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(len(vocab), output_dim=embedding_dim, embeddings_regularizer=keras.regularizers.l2(.001))(x_v)
x = LSTM(128, dropout=0.3,recurrent_dropout=0.2, return_sequences=True)(emb)
x = MyAttention()(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer=keras.optimizers.AdamW(), loss=keras.losses.BinaryCrossentropy(from_logits=False), metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best.weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

In [ ]:
batch_size = 32
epochs = 20

history = model.fit(X_train, y_train.values, validation_split=0.2, callbacks=[model_checkpoint_callback], epochs=epochs, batch_size=batch_size)

show_history(history)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test.values)
test_acc

# ✅  Tasks for the lecture (2p)

* `Attention` layer from the [library](https://github.com/philipperemy/keras-attention) has 2 `score` variants (1p)
    * Use the layer in your model and test both `score` variants
    * Is there any difference in the performance?

* It is possible to make LSTM/GRU layers `Bidirectional` using the [Bidirectional layer
](https://keras.io/api/layers/recurrent_layers/bidirectional/) (1p)
    * Use it in your model - what happened to the number of weights? Was there any improvement?